<a href="https://colab.research.google.com/github/azhar3339/Data-Analytics-Udacity/blob/master/falcon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade transformers
!pip install accelerate
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 28.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Usi

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

In [2]:
model_id = "tiiuae/falcon-11B"

# use quantization to lower GPU usage
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    quantization_config=bnb_config
)
model.generation_config.pad_token_id = model.generation_config.eos_token_id

# terminators = [
#     tokenizer.eos_token_id
# ]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [3]:
text1 = """
"Diabetes is a chronic condition where the body either cannot produce enough insulin or cannot effectively use the insulin it produces. Type 1 diabetes is an autoimmune condition where the immune system destroys insulin-producing cells in the pancreas, requiring lifelong insulin injections. Type 2 diabetes, more common, occurs when the body becomes resistant to insulin or doesn't produce enough of it. Management typically involves lifestyle changes, such as a balanced diet, regular exercise, and monitoring blood sugar levels. Medications, including insulin or oral drugs, may also be needed.

Effective diabetes management is crucial to prevent complications like heart disease, kidney damage, and nerve issues. Regular monitoring of blood sugar levels, adherence to prescribed medications, and maintaining a healthy weight are key to controlling the condition. Lifestyle modifications, such as a nutritious diet and stress management, play a significant role in managing diabetes and improving overall health. Consulting with a healthcare provider for personalized advice and regular check-ups is essential for optimal management.
"""
# text1 = """
# "Hypertension, or high blood pressure, is a condition where the force of blood against the walls of the arteries is consistently too high. This increased pressure makes the heart work harder and can lead to damage of the blood vessels over time. Blood pressure is measured in millimeters of mercury (mmHg) and is recorded with two numbers: systolic (the pressure when the heart beats) over diastolic (the pressure when the heart rests between beats). A reading consistently above 130/80 mmHg is generally considered high.

# If left unmanaged, hypertension can lead to serious health problems, including heart disease, stroke, and kidney damage. Managing hypertension involves lifestyle changes such as a healthy diet low in sodium, regular physical activity, and maintaining a healthy weight. Medications may also be prescribed to help control blood pressure. Regular monitoring and working with a healthcare provider are essential for managing the condition and reducing the risk of complications.
# """
# text1 = """
# "Asthma is a chronic respiratory condition characterized by inflammation and narrowing of the airways, making it difficult to breathe. It is often triggered by factors such as allergens, respiratory infections, exercise, or environmental pollutants. Common symptoms include wheezing, shortness of breath, chest tightness, and coughing, particularly at night or early in the morning. The severity of symptoms can vary from mild to life-threatening.

# Effective management of asthma involves avoiding known triggers, using medications such as inhaled corticosteroids and bronchodilators, and monitoring symptoms regularly. Asthma action plans, developed with a healthcare provider, help individuals manage their condition by outlining steps to take during worsening symptoms or asthma attacks. With proper treatment and management, most people with asthma can lead a normal, active life.
# """
# text1 = """
# "Heart disease encompasses a range of conditions that affect the heart and blood vessels. The most common type is coronary artery disease (CAD), where the coronary arteries become narrowed or blocked due to plaque buildup, leading to reduced blood flow to the heart muscle. This can result in chest pain (angina) or heart attacks. Other forms of heart disease include heart failure, where the heart is unable to pump blood effectively, and arrhythmias, which are irregular heartbeats.

# Risk factors for heart disease include high blood pressure, high cholesterol, smoking, obesity, diabetes, and a sedentary lifestyle. Managing heart disease involves lifestyle changes like a heart-healthy diet, regular exercise, and quitting smoking, as well as medications to control risk factors and improve heart function. Regular check-ups with a healthcare provider are crucial for monitoring and managing the condition to prevent serious complications such as heart attacks or strokes.
# """
# text1 = """
# "A stroke occurs when blood flow to a part of the brain is interrupted, either by a blockage or a rupture of blood vessels, leading to the death of brain cells. There are two main types of stroke: ischemic and hemorrhagic. Ischemic strokes are caused by a blockage in an artery supplying blood to the brain, often due to a blood clot. Hemorrhagic strokes result from a rupture of a blood vessel, leading to bleeding in or around the brain. Both types can result in brain damage and loss of function depending on the area affected and the duration of the interruption in blood flow.

# Prompt treatment is crucial in minimizing brain damage and improving recovery outcomes. Symptoms of a stroke can include sudden numbness or weakness on one side of the body, confusion, difficulty speaking, vision problems, and severe headache. Immediate medical attention is necessary, and recovery often involves rehabilitation therapies such as physical, occupational, and speech therapy to help regain lost functions and adapt to any disabilities caused by the stroke.
# """

In [4]:
SYS_PROMPT = """You are a professional English to Arabic Translator."""
formatted_prompt = f"""Translate this to Arabic: {text1}"""
messages = [{"role":"system","content":SYS_PROMPT},{"role":"user","content":formatted_prompt}]
messages

[{'role': 'system',
  'content': 'You are a professional English to Arabic Translator.'},
 {'role': 'user',
  'content': 'Translate this to Arabic: \n"Diabetes is a chronic condition where the body either cannot produce enough insulin or cannot effectively use the insulin it produces. Type 1 diabetes is an autoimmune condition where the immune system destroys insulin-producing cells in the pancreas, requiring lifelong insulin injections. Type 2 diabetes, more common, occurs when the body becomes resistant to insulin or doesn\'t produce enough of it. Management typically involves lifestyle changes, such as a balanced diet, regular exercise, and monitoring blood sugar levels. Medications, including insulin or oral drugs, may also be needed.\n\nEffective diabetes management is crucial to prevent complications like heart disease, kidney damage, and nerve issues. Regular monitoring of blood sugar levels, adherence to prescribed medications, and maintaining a healthy weight are key to contro

In [6]:
%%time
input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

# outputs = model.generate(
#       input_ids,
#       max_new_tokens=1024,
#       eos_token_id=terminators,
#       do_sample=True,
#       temperature=0.6,
#       top_p=0.9,
#   )
outputs = model.generate(
      input_ids,
      max_new_tokens=1024,
      do_sample=True,
      temperature=0.6,
      top_p=0.9,
  )

response = outputs[0][input_ids.shape[-1]:]

text1_ar = tokenizer.decode(response, skip_special_tokens=True)

text1_ar

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


CPU times: user 2min 30s, sys: 658 ms, total: 2min 31s
Wall time: 2min 37s


'\nالدكتور: هل أعرف أنك يعجبكم أن يكون أول معرفة لك هناك؟\nمستخدم: لا أنما هناك عدد كبير من الأشياء التي تعجبني. لا أعرف أن أحدها أكثر من أخرى، لا أعرف أن أحدها أكثر من أخرى، لا أعرف أن أحدها أكثر من أخرى، لا أعرف أن أحدها أكثر من أخرى، لا أعرف أن أحدها أكثر من أخرى، لا أعرف أن أحدها أكثر من أخرى، لا أعرف أن أحدها أكثر من أخرى، لا أعرف أن أحدها أكثر من أخرى، لا أعرف أن أحدها أكثر من أخرى، لا أعرف أن أحدها أكثر من أخرى، لا أعرف أن أحدها أكثر من أخرى، لا أعرف أن أحدها أكثر من أخرى، لا أعرف أن أحدها أكثر من أخرى، لا أعرف أن أحدها أكثر من أخرى، لا أعرف أن أحدها أكثر من أخرى، لا أعرف أن أحدها أكثر من أخرى، لا أعرف أن أحدها أكثر من أخرى، لا أعرف أن أحدها أكثر من أخرى، لا أعرف أن أحدها أكثر من أخرى، لا أعرف أن أحدها أكثر من أخرى، لا أعرف أن أحدها أكثر من أخرى، لا أعرف أن أحدها أكثر من أخرى، لا أعرف أن أحدها أكثر من أخرى، لا أعرف أن أحدها أكثر من أخرى، لا أعرف أن أح'